In [4]:
from pymongo import MongoClient
import os
import sys
import pandas as pd
import numpy as np
from utils import utilsvectorizer as uv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [5]:
# Create a client connection to the MongoDb instance running on the local machine
client = MongoClient('localhost:27017')

In [6]:
# connect to database of interest
db = client.awards_tmp

In [18]:
# cursor_tmp is a generator
cursor_tmp = db.awards_tmp.find({}, {'Amount' : 1,
                                     'amount_corrected' : 1,
                                     'title' : 1,
                                     'Abstract' : 1,
                                     'Agency' : 1,
                                     'Awards Year' : 1,
                                     'Phase':1,
                                     'Branch' : 1,
                                     'company name' : 1,
                                     'Solicitation Topic Code': 1,
                                     'HUBZone Owned' : 1,
                                     'Woman Owned' : 1,
                                     'Business Contact.Title' : 1,
                                     'Research Institution': 1,
                                     'Principal Investigator.Title' : 1,
                                     '_id': False} )
# need a new cursor (generator) after you used it
df = pd.DataFrame(list(cursor_tmp))

In [19]:
# convert Awards year to datetime object
df['Awards Year'] = pd.to_datetime(df['Awards Year'])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167412 entries, 0 to 167411
Data columns (total 14 columns):
Abstract                   167412 non-null object
Agency                     167405 non-null object
Amount                     167412 non-null float64
Awards Year                167412 non-null datetime64[ns]
Branch                     82443 non-null object
Business Contact           167412 non-null object
HUBZone Owned              167412 non-null object
Phase                      167412 non-null object
Principal Investigator     167412 non-null object
Research Institution       167412 non-null object
Solicitation Topic Code    167412 non-null object
Woman Owned                167412 non-null object
company name               167412 non-null object
title                      167412 non-null object
dtypes: datetime64[ns](1), float64(1), object(12)
memory usage: 17.9+ MB


In [ ]:
# uncover nested dict
df['Principal Investigator'] = df['Principal Investigator'].apply(lambda x: x.get('Title', np.nan) )

In [33]:
df['Business Contact'] = df['Business Contact'].apply(lambda x: x.get('Title', np.nan) )

In [36]:
df['HUBZone Owned'].value_counts()

N              160797
Y                5584
Unavailable      1031
Name: HUBZone Owned, dtype: int64

In [37]:
df.loc[ df['HUBZone Owned'] == 'Unavailable'  , 'HUBZone Owned'] = np.nan
df.loc[ df['HUBZone Owned'] == 'N'  , 'HUBZone Owned'] = 0
df.loc[ df['HUBZone Owned'] == 'Y'  , 'HUBZone Owned'] = 1

In [39]:
df['Solicitation Topic Code'].value_counts().count()

14480

In [40]:
crit = df['Solicitation Topic Code'] == 'N/A'
df.loc[ crit  , 'Solicitation Topic Code'] = 0
df.loc[ ~crit  , 'Solicitation Topic Code'] = 1